<a href="https://colab.research.google.com/github/Felipehonorato1/datavisu/blob/master/chest_x_Ray/chestxray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, Dropout, BatchNormalization
from keras.callbacks import TensorBoard
import keras.layers as L
from google.colab.patches import cv2_imshow
import seaborn as sns
from keras.applications import Xception, VGG16
import matplotlib.pyplot as plt
import numpy as np
import keras
import os
import cv2

np.random.seed(1000)

In [2]:
val_path = '/content/drive/My Drive/chest_xray/val/'
train_path = '/content/drive/My Drive/chest_xray/train/'
test_path = '/content/drive/My Drive/chest_xray/test/'

In [3]:
img_shape = []
for img in os.listdir(train_path+"/"+"NORMAL/")[:10]:
  img = cv2.imread(train_path+"/"+"NORMAL/"+img, 0)
  img_shape.append(img.shape);

In [4]:
img_shape
# Conseguimos ver que as imagens estão em uma resolução alta em todo o dataset mas que não seguem um padrão. 
# Colocarei todas no formato (256,256,1)

[(1909, 2566),
 (2067, 2844),
 (2169, 2534),
 (2326, 2091),
 (1914, 1679),
 (1815, 1857),
 (2583, 2916),
 (1522, 2118),
 (2540, 2428),
 (1978, 1979)]

In [5]:
# train_gen = ImageDataGenerator()
# test_gen = ImageDataGenerator()
# val_gen = ImageDataGenerator()
# train_data = train_gen.flow_from_directory('/content/drive/My Drive/chest_xray/train', color_mode = 'grayscale', class_mode = 'binary', seed = 301)
# test_data = test_gen.flow_from_directory('/content/drive/My Drive/chest_xray/test', color_mode = 'grayscale', class_mode = 'binary', seed = 301)
# val_data = test_gen.flow_from_directory('/content/drive/My Drive/chest_xray/val', color_mode = 'grayscale', class_mode = 'binary', seed = 301)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [6]:
# print(train_data.image_shape)
# print(test_data.image_shape)
# print(val_data.image_shape)

(256, 256, 1)
(256, 256, 1)
(256, 256, 1)


In [7]:
def VGG16(weights_path = None):
  classificador = Sequential()

  classificador.add(Conv2D(64, kernel_size = (3,3), input_shape = (224,224,1), data_format = 'channels_last', activation = 'relu'))
  classificador.add(MaxPooling2D(pool_size = (2,2), strides = 2))

  classificador.add(Conv2D(128, kernel_size= (3,3), activation = 'relu'))
  classificador.add(MaxPooling2D(pool_size = (2,2), strides = 2))

  classificador.add(Conv2D(256, kernel_size = (3,3), activation = 'relu'))
  classificador.add(MaxPooling2D(pool_size = (2,2), strides = 2))

  classificador.add(Conv2D((512),kernel_size = (3,3), activation = 'relu'))
  classificador.add(MaxPooling2D(pool_size = (2,2), strides = 2))

  classificador.add(Conv2D(512, kernel_size = (3,3), activation = 'relu'))
  classificador.add(MaxPooling2D(pool_size = (2,2), strides = 2))

  classificador.add(Flatten())
  classificador.add(Dense(2048, activation = 'relu'))
  classificador.add(Dropout(0.5))
  classificador.add(Dense(1024, activation = 'relu'))
  classificador.add(Dropout(0.5))
  classificador.add(Dense(512, activation = 'relu'))
  classificador.add(Dropout(0.5))
  classificador.add(Dense(128, activation = 'relu'))
  classificador.add(Dense(1, activation = 'sigmoid'))

  if weights_path:
    classificador.load_weights(weights_path)

  classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  classificador.summary()
  return classificador

In [8]:
from keras.preprocessing import image_dataset_from_directory

In [9]:
train_data = image_dataset_from_directory(train_path,color_mode = 'grayscale', image_size = (224,224), seed = 301)
test_data = image_dataset_from_directory(test_path,color_mode = 'grayscale', image_size = (224,224), seed = 301)
val_data = image_dataset_from_directory(val_path,color_mode = 'grayscale', image_size = (224,224), seed = 301)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [10]:
vgg = VGG16()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 512)       1

In [11]:
from keras.callbacks import TensorBoard
tensor = TensorBoard()

In [ ]:
history = vgg.fit(train_data, use_multiprocessing= True, batch_size = 128, validation_data= val_data, workers = -1)

 80/163 [=============>................] - ETA: 1:50 - loss: 0.6010 - accuracy: 0.7258